# About
This notebook will produce all the final clean half-hourly energy data for step 3.2 of the data processing for Module 1.

**This requires output from the previous steps - 1.1A, 1.1B and 2 - so run those notebooks first**

---

**Required user input**

Update the cell below once each for the full years of 2019, 2020, 2021, and run the entire notebook for each.


In [ ]:
year = 2021 # Update year - this is the year of data you are working on.

In [ ]:
# Don't change these.

# Source data files from steps 1.1 and 2
source_directory_gas='Step_1_1_Outputs'
source_directory_elec=source_directory_gas
source_subdirectory_gas = 'Step_1_1A_Gas_'+str(year)+'_hh'
source_subdirectory_elec = 'Step_1_1B_Elec_'+str(year)+'_hh'
source_directory_temperature = 'Step_2_Outputs'
source_filename_temperature = 'Step_2_Temp_'+str(year)+'_hh.csv'

# Index for the year, UTC - note that this must start at 1 Jan, 00:30:00, and end the following 1 Jan, 00:00:00
index_start_date=str(year)+'-01-01 00:30:00' # Start date for the output's index to include.
index_end_date=str(year+1)+'-01-01 00:00:00' # End date for the output's index to include.

output_directory= 'Module_1_final_outputs'
output_subdirectory='hh_'+str(year)
output_filename_suffix ='_annual_report_sm_hh_'+str(year)+'.csv' # Names will start with PURPN.

# Code
## Setup

In [ ]:
import os
import pandas as pd
import numpy as np
import locations

In [ ]:
#Get list of grid_cells mapped to PUPRN, from the participant data file
puprn_to_grid_cell = pd.read_csv(os.path.join(locations.serl_data_path,locations.participant_data_file),
                                 usecols=['PUPRN','grid_cell'],
                                 index_col='PUPRN')

# Load the temperature data
temperature_data_hh = pd.read_csv(os.path.join(source_directory_temperature,source_filename_temperature),
                                 parse_dates=['date_time_utc'],
                                 index_col='date_time_utc')
temperature_data_hh.index=temperature_data_hh.index.tz_localize(tz='UTC')
temperature_data_hh.index.names=['Read_date_time_UTC']

## Process the data

## Loop through the list of PUPRNs for both fuels
We want to create a list of the final gas data first, then the final electricity data, then merge them

In [ ]:
#Get list of gas PUPRNs to work with, from the source directory
puprn_filelist_gas = [f for f in os.listdir(os.path.join(source_directory_gas,source_subdirectory_gas)) if os.path.isfile(os.path.join(source_directory_gas,source_subdirectory_gas, f))]
puprn_filelist_gas = sorted(puprn_filelist_gas, key=str.lower)
print('Check this is how many gas PUPRN hh files you were expecting to find:\n',
      len(puprn_filelist_gas))

#Get list of electricity PUPRNs to work with, from the source directory
puprn_filelist_elec = [f for f in os.listdir(os.path.join(source_directory_elec,source_subdirectory_elec)) if os.path.isfile(os.path.join(source_directory_elec,source_subdirectory_elec, f))]
puprn_filelist_elec = sorted(puprn_filelist_elec, key=str.lower)
print('Check this is how many elec PUPRN hh files you were expecting to find:\n',
      len(puprn_filelist_elec))

# Make a superlist of PUPRNs that are in at least one of the lists.
# What's in puprn_filelist_gas that's not in puprn_filelist_elec
differences = list(set(puprn_filelist_gas).difference(puprn_filelist_elec))
puprn_filelist = sorted((puprn_filelist_elec + differences), key=str.lower)
print('There are this many unique PUPRNs with at least one fuel of hh data:\n',len(puprn_filelist))

In [ ]:
# We'll be processing each PUPRN's data and saving it to separate csvs one PUPRN at a time.
puprns_saved=0
puprn_errors=[]
puprn_temp_nans=[]
puprn_out_of_sequence=[]
puprn_duplicate_rows=[]
# First, create the template index - a complete year. 30 minute time steps.
date_time_index_new = pd.date_range(index_start_date,index_end_date,freq='30T')
date_time_index_new =date_time_index_new.tz_localize(tz='UTC')
date_time_index_new

# Create the output folders if they're not already there
if not os.path.exists(os.path.join(output_directory,output_subdirectory,'Errors')):
    os.makedirs(os.path.join(output_directory,output_subdirectory,'Errors'))

for i in puprn_filelist:
    # Load data or create blank dataframes instead
    try:
        temp_data_gas = pd.read_csv(os.path.join(source_directory_gas,source_subdirectory_gas,i),
                                    usecols=['PUPRN','Read_date_time_UTC','Gas_hh_Wh'],
                                    index_col=['Read_date_time_UTC'],
                                    parse_dates=['Read_date_time_UTC'])
        temp_data_gas.index=temp_data_gas.index.tz_localize(tz='UTC')
    except: 
        temp_data_gas=pd.DataFrame(index=date_time_index_new,
                                   columns=['PUPRN','Gas_hh_Wh'])
        temp_data_gas.index=temp_data_gas.index.set_names('Read_date_time_UTC')
    
    try:
        temp_data_elec = pd.read_csv(os.path.join(source_directory_elec,source_subdirectory_elec,i),
                                     usecols=['PUPRN','Read_date_time_UTC','Elec_act_net_hh_Wh'],
                                     index_col=['Read_date_time_UTC'],
                                     parse_dates=['Read_date_time_UTC'])
        temp_data_elec.index=temp_data_elec.index.tz_localize(tz='UTC')
    except:
        temp_data_elec=pd.DataFrame(index=date_time_index_new,
                                    columns=['PUPRN','Elec_act_net_hh_Wh'])
        temp_data_elec.index=temp_data_elec.index.set_names('Read_date_time_UTC')

    # Create a blank df for the data to be joined into
    energy_data_final = pd.DataFrame(index=date_time_index_new)
    energy_data_final.index=energy_data_final.index.set_names('Read_date_time_UTC')

    # Join energy data onto it
    energy_data_final = pd.merge(energy_data_final,temp_data_elec,left_index=True,right_index=True, how='outer')
    energy_data_final = pd.merge(energy_data_final,temp_data_gas,left_index=True,right_index=True,on='PUPRN', how='outer')
    #Rename to the final column names
    energy_data_final.rename(columns={'Elec_act_net_hh_Wh':'Clean_elec_net_Wh','Gas_hh_Wh':'Clean_gas_Wh'},inplace=True)
    
    # Join temperature data onto it
    grid_cell = puprn_to_grid_cell.at[i[:-4],'grid_cell']
    temp_data_temperatures = temperature_data_hh[temperature_data_hh.grid_cell==grid_cell]
    energy_data_final = energy_data_final.join(temp_data_temperatures, how='left')
    
    # This is now comprised of clean energy data, and ready except for the local time cols, as the index is UTC.
    # Recreate the SERL-style Read_date_time_local, and read_date_effective_local
    energy_data_final['Read_date_time_local']=energy_data_final.index.tz_convert(tz='Europe/London')
    energy_data_final['Read_date_time_local_midpoint']=energy_data_final.Read_date_time_local- pd.Timedelta(minutes=15)
    energy_data_final.Read_date_time_local_midpoint=energy_data_final.Read_date_time_local_midpoint.astype('str')
    energy_data_final['Read_date_effective_local']=energy_data_final.Read_date_time_local_midpoint.str.split(None).str[0]
    energy_data_final.Read_date_time_local=energy_data_final.Read_date_time_local.astype('str')
    energy_data_final.Read_date_time_local=energy_data_final.Read_date_time_local.replace({'\+00:00':' GMT'},regex=True)
    energy_data_final.Read_date_time_local=energy_data_final.Read_date_time_local.replace({'\+01:00':' BST'},regex=True)

    # Create Readings_from_midnight_local
    energy_data_final['Read_time_local']=energy_data_final.Read_date_time_local.str.split(None).str[1]
    energy_data_final['Read_time_local_midpoint']=(pd.to_datetime(energy_data_final.Read_time_local,format="%H:%M:%S") 
                                  - pd.Timedelta(minutes=15))
    energy_data_final['Readings_from_midnight_local'] = (energy_data_final.Read_time_local_midpoint.dt.hour
                                          + energy_data_final.Read_time_local_midpoint.dt.minute/60)*2 +0.5

    # PUPRN is only filled for rows with energy data. This is fixed below.
    energy_data_final.PUPRN = i[:-4]
    
    #For 2021 only, the last data point of temperature data is missing as it is not available in the 4th Edition Obseratory data release, so we will forward fill from the previous reading.
    if year == 2021:
        energy_data_final.temp_C.fillna(method='ffill',limit=1,inplace=True)
    
    # Check for data errors, and save the relevant output (in a subfolder if there's an error)
    # - Make a note if the home has something apparently wrong with it.
    subfolder=''
    no_temp_nans = (energy_data_final.temp_C.isnull().sum()==0)
    in_sequence = energy_data_final.index.is_monotonic_increasing
    no_duplicate_rows = energy_data_final.index.is_unique
    all_rows_as_expected = (in_sequence & no_duplicate_rows & no_temp_nans)
    if energy_data_final.shape[0]!=len(date_time_index_new) or all_rows_as_expected==False:
        puprn_errors.append(i[:-4])
        if no_temp_nans == False:
            puprn_temp_nans.append(i[:-4])
        if in_sequence == False:
            puprn_out_of_sequence.append(i[:-4])
        if no_duplicate_rows == False:
            puprn_duplicate_rows.append(i[:-4])
        subfolder='Errors'
    energy_data_final[['PUPRN','Read_date_time_local','Read_date_effective_local','Readings_from_midnight_local','Clean_elec_net_Wh','Clean_gas_Wh','temp_C']].to_csv(os.path.join(output_directory,output_subdirectory,subfolder,i[:-4]+output_filename_suffix), index=True)
    puprns_saved=puprns_saved+1

    # Note progress occasionally (every 250 homes):
    if puprns_saved % 250 == 0:
        print(puprns_saved,"PUPRNs of data have been processed. Continuing...")
        if len(puprn_errors)>0:
            print("You've got",len(puprn_errors),"PUPRNs with errors so far though (saved in the subfolder 'Errors'), of which:\n",
                 len(puprn_temp_nans),"have missing temperature readings;\n",
                 len(puprn_out_of_sequence),"have out of sequence datetime rows;\n",
                 len(puprn_duplicate_rows),"have duplicate rows.")

print('\nJob done, total PUPRNs gone through =',puprns_saved,
      '\nOf which, this many had errors:',len(puprn_errors),", of which:\n",
                 len(puprn_temp_nans),"have missing temperature readings;\n",
                 len(puprn_out_of_sequence),"have out of sequence datetime rows;\n",
                 len(puprn_duplicate_rows),"have duplicate rows.")

In [ ]:
# If there are any, save list of PUPRNs with errors (pandas is actually the neatest way to save a list to csv!)
if len(puprn_errors)>0:
    pd.Series(puprn_errors).to_csv(os.path.join(output_directory,'PUPRNs_with_hh_data_errors_'+str(year)+'.csv'), index=False)
    print("\nThe full list of PUPRNs with errors is saved in the same output folder as 'PUPRNs_with_hh_data_errors_'"+str(year)+".csv'")